#### **Librerías**

In [52]:
import os
import torch
import torchaudio
import tarfile
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
import IPython
import gc
import torch.nn.functional as F
import torchaudio.transforms as tt
import matplotlib
import matplotlib.pyplot as plt
import wandb
import random
import torch.optim as optim
import torchaudio.transforms as T
import tqdm

from torch import nn, optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset, Dataset
from typing import Dict
from sklearn.metrics import accuracy_score, confusion_matrix
from torch.utils.data import Dataset
from torchaudio.datasets import GTZAN
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch.optim.lr_scheduler import ReduceLROnPlateau
import collections
from torchaudio.transforms import MelSpectrogram, AmplitudeToDB
from torchaudio.sox_effects import apply_effects_tensor
from typing import Optional

%matplotlib inline

#### **Funciones**

In [2]:
def parse_genres(fname):
    parts = fname.split('/')[-1].split('.')[0]
    return parts #' '.join(parts[0])

#### **Clases**

In [42]:
class MusicDataset(Dataset):
    def __init__(self, root = './genres_5sec/'):
        super().__init__()
        self.root = root
        self.files =[]
        classes = self.get_classes(self.root)
        for c in classes:
          self.files = self.files + [fname for fname in os.listdir(os.path.join(root,c)) if fname.endswith('.wav')]
        self.classes = list(set(parse_genres(fname) for fname in self.files))

    def __len__(self):
        return len(self.files)

    def __getitem__(self, i):
        fname = self.files[i]
        genre = parse_genres(fname)
        fpath = os.path.join(self.root,genre, fname)
        class_idx = self.classes.index(genre)
        audio = torchaudio.load(fpath)[0]

        return audio, class_idx

    def get_classes(self, root):
        list_files = os.listdir(root)
        classes = []
        for file in list_files:
            name = '{}/{}'.format(root, file)
            if os.path.isdir(name):
                classes.append(file)
        return classes

# [(W−K+2P)/S]+1, where W is the input size, K is the kernel size, P is the padding, and S is the stride.
class CELAutoenconder(nn.Module):
    def __init__(self):
        super(CELAutoenconder, self).__init__()
        
        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv1d(1, 16, kernel_size=9, stride=1, padding=4), # Input size = 110250 -> Output size = 110250 (formula: [(110250-9+2*4)/1]+1 = [(110250-9+8)/1]+1 = [110250/1]+1 = 110250+1 = 110251) -> 16x110250
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2), # Input size = 110250 -> Output size = 110249 (formula: [(110250-2)/2]+1 = [(110248)/2]+1 = 110248/2+1 = 55124+1 = 55125) -> 16x55125
            
            nn.Conv1d(16, 32, kernel_size=9, stride=1, padding=4), # Input size = 55125 -> Output size = 55125 -> 32x55125
            nn.ReLU(),
            
            nn.Conv1d(32, 64, kernel_size=9, stride=1, padding=4), # Input size = 55125 -> Output size = 55125 -> 64x55125
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2), # Input size = 55125 -> Output size = 27562 -> 64x27562
            
            nn.Conv1d(64, 128, kernel_size=9, stride=1, padding=4), # Input size = 27562 -> Output size = 27562 -> 128x27562
            nn.ReLU(),

            nn.Conv1d(128, 256, kernel_size=9, stride=1, padding=4), # Input size = 27562 -> Output size = 27562 -> 256x27562
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2), # Input size = 27562 -> Output size = 13781 -> 256x13781

            nn.Conv1d(256, 512, kernel_size=9, stride=1, padding=4), # Input size = 13781 -> Output size = 13781 -> 512x13781
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2), # Input size = 13781 -> Output size = 6890 -> 512x6890
        )
        
        # Decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose1d(512, 256, kernel_size=9, stride=2, padding=4, output_padding=1), # Input size = 6890 -> Output size = 13781 -> 256x13781
            nn.ReLU(),
            nn.Upsample(size=27562),  # Tamaño específico en lugar de scale_factor
            
            nn.ConvTranspose1d(256, 128, kernel_size=9, stride=1, padding=4), # Input size = 13781 -> Output size = 27562 -> 128x27562
            nn.ReLU(),
            nn.ConvTranspose1d(128, 64, kernel_size=9, stride=1, padding=4), # Input size = 27562 -> Output size = 55125 -> 64x27562
            nn.ReLU(),
            nn.Upsample(size=55125),  # Tamaño específico
            
            nn.ConvTranspose1d(64, 32, kernel_size=9, stride=1, padding=4), # Input size = 55125 -> Output size = 110250 -> 32x55125
            nn.ReLU(),
            nn.ConvTranspose1d(32, 16, kernel_size=9, stride=1, padding=4), # Input size = 55125 -> Output size = 110250 -> 16x55125
            nn.ReLU(),
            nn.Upsample(size=110250),  # Tamaño específico para match exacto
            
            nn.ConvTranspose1d(16, 1, kernel_size=9, stride=1, padding=4), # Input size = 110250 -> Output size = 220500 -> 1x110250
            nn.Sigmoid()
        )
        
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        
        if decoded.size(2) != x.size(2):
            decoded = F.interpolate(decoded, size=x.size(2), mode='linear')
        return decoded

In [ ]:
samplerate = 22050

random_seed = 43992294

torch.manual_seed(random_seed);

if torch.cuda.is_available():
    torch.cuda.manual_seed(random_seed)

device = torch.device('cuda:0' if torch.cuda.is_available() else
                    'mps' if torch.backends.mps.is_available() else
                    'cpu')

print(f"Dispositivo usado: {device}")
print(f"CUDA disponible: {torch.cuda.is_available()}")
print(f"MPS disponible: {torch.backends.mps.is_available()}")

In [44]:
dataset = MusicDataset()

#### **Visualización**

In [ ]:
waveform, label = dataset[1]
print("shape of waveform {}, sample rate with {}, label is {} ".format(waveform.size(),samplerate,label))

In [ ]:
specgram=tt.Spectrogram()(waveform)
print("shape of spectogram {}".format(specgram.size()))

plt.figure(figsize=(20, 5))
plt.imshow(specgram.log2()[0,:,:].numpy(),cmap='magma')

In [ ]:
print("Waveform: {}\n".format(waveform))

plt.figure()
plt.plot(waveform.t().numpy())

In [ ]:
IPython.display.Audio(waveform,rate=samplerate)

#### **Experimentos**


In [49]:
batch_size = 20
val_size = 100
test_size = 100
train_size = len(dataset) - val_size - test_size

train_ds, val_ds, test_ds = random_split(dataset, [train_size, val_size, test_size])

if torch.cuda.is_available():
    train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=4, pin_memory=True)
    valid_dl = DataLoader(val_ds, batch_size*2, num_workers=4, pin_memory=True)
    test_dl = DataLoader(test_ds,1, num_workers=4, pin_memory=True)
else:
    train_dl = DataLoader(train_ds, batch_size, shuffle=True)
    valid_dl = DataLoader(val_ds, batch_size*2)
    test_dl = DataLoader(test_ds,1)

In [ ]:
# Inicialización del modelo
autoencoder = CELAutoenconder().to(device)

# Parámetros del modelo
parameters = [p for p in autoencoder.parameters() if p.requires_grad]

# Optimizador
optimizer = optim.Adam(parameters, lr=0.001)

# Obtener un lote de datos de entrenamiento
wav, label = next(iter(train_dl))
print("Forma de los datos antes de moverlos al dispositivo:", wav.shape)

# Mover los datos al dispositivo seleccionado
wav = wav.to(device)

# Imprimir la forma de los datos
print("Forma de los datos después de moverlos al dispositivo:", wav.shape)

In [ ]:
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(autoencoder.parameters(), lr=0.001)
valid_losses = []
num_epochs = 5

torch.cuda.empty_cache()
gc.collect() #importante para ir liberando memoria ram

for epoch in range(num_epochs):
    losses = []

    # Train
    autoencoder.train()
    for wav, _ in train_dl:
        optimizer.zero_grad()  # Clear gradients
        wav=wav.to(device)

        # Forward
        out = autoencoder(wav)

        # Slice the last 23 points
        loss = loss_function(out, wav)

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        losses.append(loss.item())

        del wav 
        del loss 
        del out 
        torch.cuda.empty_cache()  
        gc.collect() 

    print('Epoch: [%d/%d], Train loss: %.4f' % (epoch+1, num_epochs, np.mean(losses)))

In [ ]:
wav, _ = next(iter(train_dl))

IPython.display.Audio(wav[1],rate=samplerate)

In [ ]:
prediction = autoencoder(wav.to(device)).detach()

IPython.display.Audio(prediction[1].cpu(),rate=samplerate)

In [ ]:
# Create a figure with two subplots vertically stacked
plt.figure(figsize=(12, 6))

# Plot original waveform
plt.subplot(2, 1, 1)
plt.plot(wav[0, 0, :50].cpu().numpy())
plt.title('Original Waveform')
plt.ylabel('Amplitude')
plt.grid(True)

# Plot reconstructed waveform 
plt.subplot(2, 1, 2)
plt.plot(prediction[0, 0, :50].cpu().numpy())
plt.title('Reconstructed Waveform')
plt.xlabel('Sample')
plt.ylabel('Amplitude')
plt.grid(True)

# Adjust layout to prevent overlap
plt.tight_layout()
plt.show()

In [ ]:
val_loss = 0
val_acc = 0
train_acc = 0

lr = 0.0001
best_valid_loss = float('inf')  # Para trackear el mejor modelo
best_state_dict = None  # Para guardar el mejor estado del modelo

loss_function = nn.MSELoss()

train_losses = []
train_accs = []
valid_losses = []
valid_accs = []
num_epochs = 30


iterator = tqdm(range(num_epochs), total=num_epochs, desc="Epoch")

for epoch in iterator:
    train_losses_itter = []

    total = 0
    train_correct = 0

    # Train
    autoencoder.train()
    for wav, _, spectogram, genre_index in train_dl:
        
        optimizer.zero_grad()
        wav = wav.to(device)
        genre_index = torch.as_tensor(genre_index).to(device)

        # Forward
        out = autoencoder(wav)
        
        loss = loss_function(out.squeeze(), wav)

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_losses_itter.append(loss.item())
        
        pred = out.argmax(dim=-1).flatten()
        train_correct += (pred == genre_index).sum().item()
        total += len(pred)
        train_acc = 100 * train_correct / total

        del wav         #importante para ir liberando memoria ram
        del genre_index #importante para ir liberando memoria ram
        del loss        #importante para ir liberando memoria ram
        del out         #importante para ir liberando memoria ram
        
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        elif torch.backends.mps.is_available():
            torch.mps.empty_cache()

        gc.collect() 

        train_losses.append(np.mean(train_losses_itter))

    #print('Epoch: [%d/%d], Train loss: %.4f' % (epoch+1, num_epochs, np.mean(losses)))

    # Validation
    autoencoder.eval()
    y_true = []
    y_pred = []
    val_losses_itter = []
    correct =0
    for wav, _, spectogram, genre_index in valid_dl:

        wav = wav.to(device)
        genre_index = genre_index.to(device)

        out = autoencoder(wav)

        loss = loss_function(out.squeeze(), genre_index)

        val_losses_itter.append(loss.item())

        pred = out.argmax(dim=-1).flatten()
        
        correct += (pred == genre_index).sum().item()
        y_true.extend(genre_index)
        y_pred.extend(pred)

        del wav         #importante para ir liberando memoria ram
        del genre_index #importante para ir liberando memoria ram
        del loss        #importante para ir liberando memoria ram
        del out         #importante para ir liberando memoria ram
        
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        elif torch.backends.mps.is_available():
            torch.mps.empty_cache()

        gc.collect()

    val_acc = 100 * correct / len(valid_dl.dataset)
    val_loss = np.mean(val_losses_itter)

    # Save autoencoder
    train_losses.append(np.mean(train_losses))
    train_accs.append(train_acc)
    valid_losses.append(val_loss.item())
    valid_accs.append(val_acc)

    # Actualizar el mejor modelo si encontramos una mejor pérdida
    if val_loss < best_valid_loss:
        best_valid_loss = val_loss
        best_state_dict = autoencoder.state_dict().copy()
        print(f'Nuevo mejor modelo encontrado en epoch {epoch+1} con validation loss: {val_loss:.4f}')

# Al finalizar el entrenamiento, guardar el mejor modelo
print(f'Guardando el mejor modelo con validation loss: {best_valid_loss:.4f}')
torch.save(best_state_dict, 'best_model.ckpt')


#### **Evaluación**


In [ ]:
# Load the best model
S = torch.load('best_model.ckpt')
autoencoder.load_state_dict(S)

print('loaded!')

# Run evaluation
autoencoder.eval()
y_true = []
y_pred = []
correct = 0
with torch.no_grad():
    for spectogram, genre_index in test_dl:
        spectogram = spectogram.to(device)
        genre_index = genre_index.to(device)

        out = autoencoder(spectogram)

        pred = out.argmax(dim=-1).flatten()
        # append labels and predictions
        correct += pred.eq(genre_index).sum().item()
        y_true.extend(genre_index)
        y_pred.extend(pred)

accuracy = correct / len(test_dl.dataset)
print('Epoch: [%d/%d], Valid loss: %.4f, Valid accuracy: %.4f' % (epoch+1, num_epochs, val_loss, accuracy))